In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import re
import time

In [45]:
def scroll(driver, timeout, loops):
    scroll_pause_time = timeout

    last_height = driver.execute_script("return document.body.scrollHeight")
    num = 0
    
    while True and num < loops:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        num = num + 1
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [48]:
url = 'https://www.vivino.com/explore?e=eJwdijsOgCAQBW_zaqDfzo4jGGPWFQmJgAH83V5iM1PMxEIaMSRSiPyQ0UpBXhospMPi6NlvdHEJrvGOvFDhFpKvs-QzNWRaXRXcbZz6-st8itQcHw%3D%3D'
product_names = []
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0")
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
browser.get(url)

loops = 2
timeout = 5
scroll(browser, timeout, loops)

titles = browser.find_elements_by_class_name('vintageTitle__wine--U7t9G')
prices = browser.find_elements_by_xpath("(//*[contains(@class, 'addToCartButton__price')])")
ratings = browser.find_elements_by_class_name('vivinoRating__averageValue--3Navj')
#countries = browser.find_elements_by_class_name('anchor__anchor--3DOSm wineLocationHeader__country--1RcW2')
countries = browser.find_elements_by_xpath('//a[contains(@href, "/explore?country_code")]')

#for price in prices:
#    print('price:',price.text)
for title, price, rating, country in zip(titles, prices, ratings, countries):
    with open('vinData.csv', 'a', encoding='UTF8') as f:
        writer = csv.writer(f)
        date = 9999
        line = re.findall(r'\d+', title.text)
        
        if(len(line)>0):
            for l in line:
                if(len(str(l)) == 4):
                    date = l
                    
        data = [title.text, date, price.text.split("\n", 1)[0], rating.text, country.text]
        writer.writerow(data)
                                

['Estate Malbec 2018', '2018', '99', '3.8', 'Argentina']
2018


In [54]:
df = pd.read_csv('vinData.csv', header=["Name", "Year", "Price", "Rating", "Country"])
df.head()

TypeError: parser_f() got an unexpected keyword argument 'index'